# Quantum Identity-Based Signature (QIBS) with BB84 Key Exchange
This code simulates a Quantum Identity-Based Signature (QIBS) scheme, a sophisticated method for verifying the authenticity and integrity of a digital message using quantum mechanics. The simulation integrates two core quantum algorithms: the BB84 protocol for secure key distribution and a quantum signature protocol where a user's identity is directly used for verification. The purpose is to demonstrate a complete and secure workflow from key generation to signature verification.

The simulation follows these key steps:

## Step 1: Establishing a Secure Key with the BB84 Protocol
The process begins with two parties, Alice (the sender) and Bob (the receiver), generating a shared secret key using the BB84 Quantum Key Distribution (QKD) protocol. This ensures that their communication key is secure from eavesdroppers.

- Preparation: Alice generates a random string of classical bits and a random string of quantum bases (either the computational Z basis {∣0⟩,∣1⟩} or the Hadamard X basis {∣+⟩,∣−⟩}). She encodes her bits onto qubits according to her chosen bases.

- Measurement: Bob receives the qubits and measures each one using his own randomly chosen string of bases (Z or X).

- Sifting: Alice and Bob publicly compare their basis choices for each qubit. They discard all measurements where their bases did not match. The remaining, correctly-measured bits form their shared, sifted key.

- Security Check: They compare a small subset of their sifted keys to calculate the Quantum Bit Error Rate (QBER). A QBER above a certain threshold (e.g., 11%) indicates the presence of an eavesdropper, and the key is discarded.

## Step 2: User Registration with a Trusted Authority
A central, trusted entity called the Secret Key Generator (SKG) is responsible for managing user identities.

The sender, Alice, is registered with the SKG. The SKG stores two secret pieces of information linked to her identity:

The shared key generated via the BB84 protocol.

A unique secret angle ϕ, which serves as her quantum identity.

## Step 3: Signature Creation by the Sender (Alice)
To sign a classical message (e.g., 32191, which is 111110110101111 in binary), Alice creates a quantum signature package consisting of two quantum states.

- Message State ∣P⟩: She first encodes the classical binary message into a multi-qubit quantum state. For each 1 in the message, she applies an X gate to the corresponding qubit. This creates the original message state ∣P⟩.

- Signature State ∣S⟩: This state is created in two steps:

- Identity Transformation: Alice applies a unique unitary transformation U(π/2,ϕ,0) to every qubit in the message state ∣P⟩. This transformation is parameterized by her secret identity angle ϕ. This mathematically binds her identity to the message.

- Encryption: She then encrypts the transformed state using the Quantum One-Time Pad (QOTP). This is done by applying Z and X gates to the qubits based on the bits of the shared BB84 key, making the state unreadable without the key.

## Step 4: Signature Verification by the Recipient and SKG
To verify the signature, the recipient (Bob) forwards the signature package (∣P⟩ and ∣S⟩) along with Alice's ID to the SKG. The SKG performs the verification by reversing Alice's steps.

- Information Retrieval: The SKG looks up Alice's ID in its registry to retrieve her secret key and secret angle ϕ.

- Decryption: It applies the inverse of the QOTP to the signature state ∣S⟩ using the retrieved key.

- Inverse Transformation: It then applies the inverse of the identity transformation using the retrieved angle ϕ.

- Comparison: If the signature is authentic and untampered, this process will perfectly recover the original message state ∣P⟩. The SKG compares the state vector of the recovered state with the original ∣P⟩. If they are identical, the signature is valid. Otherwise, it is invalid.

## Step 5: Tampering Detection
The simulation includes a final security test where an X gate is applied to a random qubit in the signature state ∣S⟩ to simulate tampering. The verification process is run again, and it correctly identifies the signature as invalid, demonstrating the scheme's robustness against modification.

## Summary:
This simulation demonstrates the core principles of an advanced quantum-secure communication scheme:

- Provable Security: The key exchange is secured by the fundamental principles of quantum mechanics (like the no-cloning theorem) through the BB84 protocol.

- Identity-Based Cryptography: A user's identity is directly used as a cryptographic parameter (the angle ϕ), simplifying key management compared to traditional public-key systems.

- Authentication and Integrity: The verification process guarantees that the message was signed by the legitimate sender (authentication) and has not been altered in transit (integrity).

- Confidentiality: The Quantum One-Time Pad ensures that the signature state remains confidential during transmission.






In [3]:
import math
import random
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

# --- Configuration ---
SENSOR_DATA = 32191 # Dummy sensor data for signing

# --- Trusted Authority: Secret Key Generator ---
class SKG:
    """A trusted entity to generate and store user secrets."""
    def __init__(self):
        self.user_registry = {} # Stores keys and secrets for each user
        print("SKG instance created.")

    def register_user(self, user_id: str, key: list, secret_angle: float):
        """Manually stores a user's key and secret angle."""
        self.user_registry[user_id] = {'key': key, 'phi': secret_angle}
        print(f"User '{user_id}' registered with SKG.")

# --- Quantum Protocols ---
def bb84_protocol(num_bits=200) -> list:
    """
    Corrected implementation of BB84 protocol to generate a shared secret key.
    Returns the sifted key with proper state preparation and QBER calculation.
    """
    print(f"Starting BB84 protocol with {num_bits} qubits")
    
    # Alice generates random bits and bases (0=Z, 1=X)
    alice_bits = np.random.randint(2, size=num_bits)
    alice_bases = np.random.randint(2, size=num_bits)

    # Bob generates his random bases for measurement
    bob_bases = np.random.randint(2, size=num_bits)

    print(f"Alice bits: {alice_bits[:20]}... (showing first 20)")
    print(f"Alice bases: {alice_bases[:20]}... (0=Z, 1=X)")
    print(f"Bob bases: {bob_bases[:20]}...")

    # Alice prepares qubits CORRECTLY according to BB84 protocol
    qc = QuantumCircuit(num_bits, num_bits)
    for i in range(num_bits):
        # Correct BB84 state preparation
        if alice_bits[i] == 1:
            if alice_bases[i] == 0:  # Base Z
                qc.x(i)  # |1⟩
            else:  # Base X
                qc.x(i)
                qc.h(i)  # |-⟩
        else:  # alice_bits[i] == 0
            if alice_bases[i] == 1:  # Base X
                qc.h(i)  # |+⟩
            # Base Z: |0⟩ (default state)
    
    qc.barrier()

    # Bob measures the incoming qubits in his chosen bases
    for i in range(num_bits):
        if bob_bases[i] == 1:  # X basis measurement
            qc.h(i)
        qc.measure(i, i)

    # Execute the circuit using Qiskit Aer simulator
    backend = AerSimulator()
    counts = backend.run(qc, shots=1).result().get_counts(qc)
    bob_results = [int(bit) for bit in list(counts.keys())[0]][::-1]

    print(f"Bob measurements: {bob_results[:20]}... (showing first 20)")

    # Sifting: Alice and Bob compare bases and keep matching bits
    sifted_key = []
    alice_sifted = []
    bob_sifted = []
    
    print("\nSifting process (keeping only matching bases):")
    for i in range(num_bits):
        if alice_bases[i] == bob_bases[i]:
            sifted_key.append(alice_bits[i])
            alice_sifted.append(alice_bits[i])
            bob_sifted.append(bob_results[i])

    print(f"Matching bases found: {len(sifted_key)}/{num_bits}")

    # CORRECTED QBER calculation using only sifted bits
    if len(alice_sifted) > 0:
        errors = sum(1 for i in range(len(alice_sifted)) if alice_sifted[i] != bob_sifted[i])
        qber = errors / len(alice_sifted)
    else:
        qber = 1.0

    # Security thresholds for BB84
    if qber > 0.25:  # Critical threshold
        print(f"CRITICAL: QBER of {qber:.3f} exceeds maximum acceptable limit (25%). Key rejected.")
        return []
    elif qber > 0.11:  # Standard security threshold
        print(f"Warning: QBER of {qber:.3f} above ideal threshold (11%). Possible eavesdropping.")
    else:
        print(f"Good: QBER of {qber:.3f} within secure range.")

    print(f"BB84 completed. Generated key with {len(sifted_key)} bits (QBER: {qber:.3f}).")
    return sifted_key

def apply_qotp(qc: QuantumCircuit, target_qubits: range, key: list):
    """Encrypts qubits using the Quantum One-Time Pad."""
    key_len_needed = 2 * len(target_qubits)
    if len(key) < key_len_needed:
        raise ValueError(f"QOTP key is too short. Needs {key_len_needed}, got {len(key)}.")

    for i, qubit_idx in enumerate(target_qubits):
        if key[2 * i] == 1: qc.z(qubit_idx)
        if key[2 * i + 1] == 1: qc.x(qubit_idx)

def apply_qotp_inverse(qc: QuantumCircuit, target_qubits: range, key: list):
    """Decrypts qubits by applying the inverse QOTP operation."""
    key_len_needed = 2 * len(target_qubits)
    if len(key) < key_len_needed:
        raise ValueError(f"QOTP key is too short. Needs {key_len_needed}, got {len(key)}.")
        
    for i, qubit_idx in enumerate(target_qubits):
        # Inverse is applied in reverse order
        if key[2 * i + 1] == 1: qc.x(qubit_idx)
        if key[2 * i] == 1: qc.z(qubit_idx)

# --- QIBS Core Logic ---
def create_signature(message_binary: str, sender_id: str, key: list, phi_angle: float):
    """
    Alice's signing process 
    Creates the original message state |P> and the signed state |S>.
    """
    num_qubits = len(message_binary)
    
    # Verify key length before proceeding
    key_len_needed = 2 * num_qubits
    if len(key) < key_len_needed:
        raise ValueError(f"Key too short for message. Need {key_len_needed} bits, got {len(key)}.")

    # 1. Prepare the message state |P>
    qc_P = QuantumCircuit(num_qubits, name="P_state")
    for i, bit in enumerate(message_binary):
        if bit == '1':
            qc_P.x(i)

    # 2. Create the signature circuit |S> by transforming and encrypting |P>
    qc_S = qc_P.copy(name="S_state")
    
    # Apply the sender's unique unitary transformation
    for i in range(num_qubits):
        qc_S.u(math.pi / 2, phi_angle, 0, i)
    
    # Encrypt with QOTP using the shared key
    apply_qotp(qc_S, range(num_qubits), key[:key_len_needed])

    print(f"Signature created by '{sender_id}' for message |{message_binary}⟩.")
    print(f"Message length: {num_qubits} qubits, Key used: {key_len_needed} bits")
    return qc_P, qc_S

def verify_signature(qc_P, qc_S, signer_id: str, skg: SKG) -> bool:
    """
    Bob and SKG's verification process.
    SKG uses its records to decrypt |S> and checks if it matches |P>.
    """
    # Retrieve signer's info from the trusted SKG
    if signer_id not in skg.user_registry:
        print(f"Verification failed: Signer '{signer_id}' not found in registry.")
        return False
    
    signer_info = skg.user_registry[signer_id]
    retrieved_key = signer_info['key']
    retrieved_phi = signer_info['phi']

    try:
        # Verify key length is sufficient
        key_len_needed = 2 * qc_S.num_qubits
        if len(retrieved_key) < key_len_needed:
            print(f"Verification failed: Insufficient key length. Need {key_len_needed}, got {len(retrieved_key)}.")
            return False

        # SKG attempts to recover the original state |P'> from the signature |S>
        qc_recovered = qc_S.copy(name="recovered_P")
        
        # Step 1: Decrypt with QOTP inverse
        apply_qotp_inverse(qc_recovered, range(qc_S.num_qubits), retrieved_key[:key_len_needed])
        
        # Step 2: Apply the inverse unitary transformation
        for i in range(qc_S.num_qubits):
            qc_recovered.u(math.pi / 2, retrieved_phi, 0, i).inverse()

        # Step 3: Compare the statevectors of the original and recovered circuits
        original_sv = Statevector(qc_P)
        recovered_sv = Statevector(qc_recovered)
        
        # Use numerical tolerance for quantum state comparison
        is_valid = original_sv.equiv(recovered_sv, rtol=1e-10, atol=1e-10)
        
        if is_valid:
            print("Verification successful: Recovered state matches original.")
        else:
            print("Verification FAILED: States do not match. Signature is invalid.")
            
        return is_valid
        
    except Exception as e:
        print(f"Verification error: {e}")
        return False

# --- Main Execution ---
if __name__ == "__main__":
    print("\n=== Quantum Identity-Based Signature (QIBS) Simulation ===")
    print("Integrating BB84 QKD with Quantum Digital Signatures")

    # 1. Initialization
    skg = SKG()
    ALICE_ID = "Carbon_Sensor"
    BOB_ID = "Regulator_Agency"
    
    print("\n--- Step 1: Establishing shared key via BB84 ---")
    # Generate shared key with  BB84 implementation
    shared_key = bb84_protocol(num_bits=300)
    
    if not shared_key:
        print("ERROR: Failed to establish secure key via BB84. Aborting simulation.")
        print("Try running again.")
        exit()
    
    # Convert sensor data to binary message
    message = bin(SENSOR_DATA)[2:]
    key_needed = 2 * len(message)
    
    print(f"\n--- Step 2: Checking key sufficiency ---")
    print(f"Message length: {len(message)} bits")
    print(f"Key needed for QOTP: {key_needed} bits")
    print(f"Available key length: {len(shared_key)} bits")

    if len(shared_key) < key_needed:
        print(f"\nERROR: Insufficient key length for message encryption.")
        print(f"Need {key_needed} bits, but only {len(shared_key)} available.")
        print("Try running again to get a longer key from BB84.")
        exit()

    # Register users with SKG
    alice_secret_phi = random.uniform(0, 2 * math.pi)
    skg.register_user(ALICE_ID, shared_key[:key_needed], alice_secret_phi)
    skg.register_user(BOB_ID, shared_key[:key_needed], 0)  # Bob doesn't need secret angle
    print("-" * 60)

    print(f"\n--- Step 3: Alice creates quantum signature ---")
    print(f"Message to sign: {SENSOR_DATA} (binary: {message})")
    
    try:
        # Alice signs the message using QDS
        qc_P, qc_S = create_signature(message, ALICE_ID, shared_key[:key_needed], alice_secret_phi)
        print("-" * 60)
        
        print(f"\n--- Step 4: Bob verifies the signature ---")
        # Bob receives the signature package (|P> and |S>) and verifies it with SKG
        print("Bob received signature package and is now verifying...")
        is_valid_signature = verify_signature(qc_P, qc_S, ALICE_ID, skg)
        print(f"\nFINAL RESULT: Signature is {'✓ VALID' if is_valid_signature else '✗ INVALID'}")
        print("-" * 60)

        print(f"\n--- Step 5: Security Test (Tampering Detection) ---")
        
        print("SECURITY TEST: Simulating signature tampering...")
        qc_S_tampered = qc_S.copy()
        tamper_qubit_index = random.randint(0, len(message) - 1)
        qc_S_tampered.x(tamper_qubit_index)
        print(f"Applied X-gate to qubit {tamper_qubit_index} to simulate tampering.")
        
        print("Bob verifying tampered signature...")
        is_tampered_valid = verify_signature(qc_P, qc_S_tampered, ALICE_ID, skg)
        
        if not is_tampered_valid:
            print("✓ SECURITY TEST PASSED: Tampered signature correctly rejected.")
        else:
            print("✗ SECURITY BREACH: System incorrectly accepted tampered signature!")
            
        print("-" * 60)
        
        print(f"\n=== SIMULATION SUMMARY ===")
        print(f"BB84 Key Generation: ✓ Success ({len(shared_key)} bits)")
        print(f"QDS Signature Creation: ✓ Success")
        print(f"Valid Signature Verification: {'✓' if is_valid_signature else '✗'}")
        print(f"Tampered Signature Detection: {'✓' if not is_tampered_valid else '✗'}")
        print(f"Overall Security: {'SECURE' if is_valid_signature and not is_tampered_valid else 'COMPROMISED'}")
        
    except Exception as e:
        print(f"ERROR in signature process: {e}")
        print("Simulation aborted due to error.")


=== Quantum Identity-Based Signature (QIBS) Simulation ===
Integrating BB84 QKD with Quantum Digital Signatures
SKG instance created.

--- Step 1: Establishing shared key via BB84 ---
Starting BB84 protocol with 300 qubits
Alice bits: [1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 1]... (showing first 20)
Alice bases: [1 1 0 1 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0]... (0=Z, 1=X)
Bob bases: [0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1]...
Bob measurements: [1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0]... (showing first 20)

Sifting process (keeping only matching bases):
Matching bases found: 151/300
Good: QBER of 0.000 within secure range.
BB84 completed. Generated key with 151 bits (QBER: 0.000).

--- Step 2: Checking key sufficiency ---
Message length: 15 bits
Key needed for QOTP: 30 bits
Available key length: 151 bits
User 'Carbon_Sensor' registered with SKG.
User 'Regulator_Agency' registered with SKG.
------------------------------------------------------------

--- Step 3: Alice cre